# MICE (Multiple Imputation by Chained Equations)
En este notebook se implementa el algoritmo MICE (Multiple Imputation by Chained Equations). Se utiliza regresión lineal simple para imputar valores faltantes de manera iterativa.

In [1]:
# Crear un DataFrame de ejemplo con valores faltantes
import numpy as np
import pandas as pd

data = {
    'A': [1, 2, np.nan, 4, 5],
    'B': [5, np.nan, np.nan, 8, 10],
    'C': [10, 11, 12, np.nan, 15]
}
df = pd.DataFrame(data)
print('Datos originales:')
print(df)

Datos originales:
     A     B     C
0  1.0   5.0  10.0
1  2.0   NaN  11.0
2  NaN   NaN  12.0
3  4.0   8.0   NaN
4  5.0  10.0  15.0


In [2]:
# Función para ajustar una regresión lineal simple y predecir valores faltantes
def linear_regression_impute(X, y):
    # X: matriz de predictores (sin valores faltantes)
    # y: variable objetivo (con valores faltantes)
    mask = ~np.isnan(y)
    X_train = X[mask]
    y_train = y[mask]
    # Ajuste de regresión lineal: y = a*X + b
    a, b = np.polyfit(X_train.flatten(), y_train, 1)
    # Predecir valores faltantes
    y_pred = a * X.flatten() + b
    return y_pred

In [3]:
# Implementación básica de MICE manual con regresión lineal simple
df_mice = df.copy()

# Inicialización: imputar con la media
for col in df_mice.columns:
    df_mice[col] = df_mice[col].fillna(df_mice[col].mean())

n_iter = 5
for it in range(n_iter):
    for col in df_mice.columns:
        other_cols = [c for c in df_mice.columns if c != col]
        # Solo imputar donde el valor original es NaN
        mask_nan = df[col].isna()
        if mask_nan.any():
            X = df_mice.loc[:, other_cols].values
            y = df_mice[col].values
            # Ajustar regresión lineal múltiple manualmente (mínimos cuadrados)
            X_train = X[~mask_nan]
            y_train = y[~mask_nan]
            # Resolver beta = (X^T X)^-1 X^T y
            X_train_aug = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
            beta = np.linalg.lstsq(X_train_aug, y_train, rcond=None)[0]
            # Predecir valores faltantes
            X_nan = X[mask_nan]
            X_nan_aug = np.hstack([X_nan, np.ones((X_nan.shape[0], 1))])
            y_pred = X_nan_aug @ beta
            df_mice.loc[mask_nan, col] = y_pred

print('Datos imputados con MICE manual:')
print(df_mice)

Datos imputados con MICE manual:
     A          B     C
0  1.0   5.000000  10.0
1  2.0   6.142857  11.0
2  3.0   7.285714  12.0
3  4.0   8.000000  12.0
4  5.0  10.000000  15.0
